https://medium.com/swlh/web-scraping-python-requests-and-beautifulsoup-45d5f48f5a1

https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/

https://medium.com/swlh/text2sql-in-spark-nlp-converting-natural-language-questions-to-sql-queries-on-scale-6ae9a9061d74

https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch04.html

In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [3]:
!pip install warcio

In [4]:
!python -m pip install cdx_toolkit

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

In [7]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|')

In [8]:
websites.head()

,0,1,2,3
0,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [9]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [10]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [11]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [12]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [13]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [14]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:10])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152418/ios-developer-w-strich-m-strich-d-fuer-die-adesso-mobile-solutions-in-stuttgart-und-karlsruhe', 'http://www.work521.com/jobs/37683.html', 'https://www.xn--itbaw-ova.de/jobs/152031/senior-consultant-w-strich-m-strich-d-sap-bca-cml-cms-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.workingin-newzealand.com/career-tools-advice/', 'https://help.workable.com/hc/en-us/articles/360053410833-Can-I-use-Google-Analytics-on-my-careers-page-', 'https:

In [15]:
statesList = ["United States of America","USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
              "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
              "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI"]
itList = ["engineer","Engineer","scientist","Scientist","developer","Developer","Analyst","analyst","Operator","operator"]
qaList = ["Qualification","Qualifications","qualification","qualifications","Certification","Certifications","certification",
          "certifications"]

In [16]:
USAJobsList = []
fout = open("./ads/2021JAN_USAjobAD.txt","w")
for i in range(0, len(urlList)):
    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            #all_USAs = re.findall('USA', fin.read())
            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
            nUSAs = len(all_USAs)
            if nUSAs > 0:
                print(str(i+1)+"_2021JAN_jobAD.txt",' ',nUSAs)
                fout.write(str(i+1)+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
                USAJobsList.append(i+1)
            else:
                continue
    else:
        continue
            
fout.close()
print(USAJobsList)

1_2021JAN_jobAD.txt   49
11_2021JAN_jobAD.txt   3
21_2021JAN_jobAD.txt   4
22_2021JAN_jobAD.txt   3
23_2021JAN_jobAD.txt   3
27_2021JAN_jobAD.txt   4
28_2021JAN_jobAD.txt   4
39_2021JAN_jobAD.txt   4
41_2021JAN_jobAD.txt   4
45_2021JAN_jobAD.txt   3
49_2021JAN_jobAD.txt   1
53_2021JAN_jobAD.txt   24
54_2021JAN_jobAD.txt   45
55_2021JAN_jobAD.txt   17
56_2021JAN_jobAD.txt   8
62_2021JAN_jobAD.txt   15
65_2021JAN_jobAD.txt   3
74_2021JAN_jobAD.txt   3
84_2021JAN_jobAD.txt   6
87_2021JAN_jobAD.txt   17
91_2021JAN_jobAD.txt   4
96_2021JAN_jobAD.txt   5
108_2021JAN_jobAD.txt   3
115_2021JAN_jobAD.txt   1
116_2021JAN_jobAD.txt   1
117_2021JAN_jobAD.txt   1
121_2021JAN_jobAD.txt   4
124_2021JAN_jobAD.txt   18
125_2021JAN_jobAD.txt   3
129_2021JAN_jobAD.txt   6
132_2021JAN_jobAD.txt   2
134_2021JAN_jobAD.txt   6
135_2021JAN_jobAD.txt   4
147_2021JAN_jobAD.txt   5
153_2021JAN_jobAD.txt   5
156_2021JAN_jobAD.txt   1
168_2021JAN_jobAD.txt   24
169_2021JAN_jobAD.txt   3
177_2021JAN_jobAD.txt   3
1

1149_2021JAN_jobAD.txt   187
1150_2021JAN_jobAD.txt   3
1152_2021JAN_jobAD.txt   52
1158_2021JAN_jobAD.txt   20
1160_2021JAN_jobAD.txt   14
1161_2021JAN_jobAD.txt   9
1163_2021JAN_jobAD.txt   14
1169_2021JAN_jobAD.txt   3
1173_2021JAN_jobAD.txt   12
1174_2021JAN_jobAD.txt   11
1176_2021JAN_jobAD.txt   11
1179_2021JAN_jobAD.txt   10
1181_2021JAN_jobAD.txt   4
1184_2021JAN_jobAD.txt   1
1193_2021JAN_jobAD.txt   21
1763_2021JAN_jobAD.txt   11
1773_2021JAN_jobAD.txt   10
1776_2021JAN_jobAD.txt   4
1777_2021JAN_jobAD.txt   6
1782_2021JAN_jobAD.txt   8
1783_2021JAN_jobAD.txt   11
1784_2021JAN_jobAD.txt   11
1785_2021JAN_jobAD.txt   4
1786_2021JAN_jobAD.txt   4
1788_2021JAN_jobAD.txt   5
1789_2021JAN_jobAD.txt   3
1791_2021JAN_jobAD.txt   11
1793_2021JAN_jobAD.txt   4
1794_2021JAN_jobAD.txt   10
1796_2021JAN_jobAD.txt   1
1797_2021JAN_jobAD.txt   28
1798_2021JAN_jobAD.txt   7
1800_2021JAN_jobAD.txt   7
1802_2021JAN_jobAD.txt   2
1806_2021JAN_jobAD.txt   2
1807_2021JAN_jobAD.txt   11
1808_2021

2771_2021JAN_jobAD.txt   11
2773_2021JAN_jobAD.txt   20
2775_2021JAN_jobAD.txt   4
2776_2021JAN_jobAD.txt   41
2777_2021JAN_jobAD.txt   2
2780_2021JAN_jobAD.txt   12
2786_2021JAN_jobAD.txt   2
2787_2021JAN_jobAD.txt   12
2788_2021JAN_jobAD.txt   44
2790_2021JAN_jobAD.txt   6
2791_2021JAN_jobAD.txt   4
2792_2021JAN_jobAD.txt   1
2796_2021JAN_jobAD.txt   7
2799_2021JAN_jobAD.txt   12
2800_2021JAN_jobAD.txt   1
2802_2021JAN_jobAD.txt   7
2803_2021JAN_jobAD.txt   1
2804_2021JAN_jobAD.txt   6
2805_2021JAN_jobAD.txt   11
2807_2021JAN_jobAD.txt   1
2810_2021JAN_jobAD.txt   6
2814_2021JAN_jobAD.txt   47
2816_2021JAN_jobAD.txt   11
2817_2021JAN_jobAD.txt   5
2819_2021JAN_jobAD.txt   14
2820_2021JAN_jobAD.txt   3
2823_2021JAN_jobAD.txt   4
2827_2021JAN_jobAD.txt   13
2830_2021JAN_jobAD.txt   23
2835_2021JAN_jobAD.txt   1
2836_2021JAN_jobAD.txt   4
2837_2021JAN_jobAD.txt   2
2838_2021JAN_jobAD.txt   49
2839_2021JAN_jobAD.txt   2
2841_2021JAN_jobAD.txt   2
2842_2021JAN_jobAD.txt   4
2843_2021JAN_j

3685_2021JAN_jobAD.txt   2
3688_2021JAN_jobAD.txt   2
3694_2021JAN_jobAD.txt   2
3700_2021JAN_jobAD.txt   20
3701_2021JAN_jobAD.txt   2
3702_2021JAN_jobAD.txt   21
3704_2021JAN_jobAD.txt   3
3706_2021JAN_jobAD.txt   6
3707_2021JAN_jobAD.txt   4
3710_2021JAN_jobAD.txt   3
3715_2021JAN_jobAD.txt   3
3717_2021JAN_jobAD.txt   20
3719_2021JAN_jobAD.txt   1
3720_2021JAN_jobAD.txt   48
3721_2021JAN_jobAD.txt   14
3723_2021JAN_jobAD.txt   1
3726_2021JAN_jobAD.txt   8
3730_2021JAN_jobAD.txt   6
3732_2021JAN_jobAD.txt   1
3740_2021JAN_jobAD.txt   5
3741_2021JAN_jobAD.txt   2
3744_2021JAN_jobAD.txt   10
3746_2021JAN_jobAD.txt   6
3747_2021JAN_jobAD.txt   2
3749_2021JAN_jobAD.txt   9
3751_2021JAN_jobAD.txt   2
3752_2021JAN_jobAD.txt   11
3754_2021JAN_jobAD.txt   12
3755_2021JAN_jobAD.txt   3
3765_2021JAN_jobAD.txt   2
3767_2021JAN_jobAD.txt   1
3769_2021JAN_jobAD.txt   15
3770_2021JAN_jobAD.txt   3
3771_2021JAN_jobAD.txt   1
3772_2021JAN_jobAD.txt   24
3773_2021JAN_jobAD.txt   53
3775_2021JAN_jobA

4713_2021JAN_jobAD.txt   6
4714_2021JAN_jobAD.txt   3
4718_2021JAN_jobAD.txt   8
4721_2021JAN_jobAD.txt   2
4722_2021JAN_jobAD.txt   1
4731_2021JAN_jobAD.txt   9
4732_2021JAN_jobAD.txt   3
4733_2021JAN_jobAD.txt   3
4734_2021JAN_jobAD.txt   3
4736_2021JAN_jobAD.txt   1
4737_2021JAN_jobAD.txt   135
4739_2021JAN_jobAD.txt   62
4746_2021JAN_jobAD.txt   3
4750_2021JAN_jobAD.txt   11
4752_2021JAN_jobAD.txt   17
4754_2021JAN_jobAD.txt   5
4762_2021JAN_jobAD.txt   14
4763_2021JAN_jobAD.txt   1
4765_2021JAN_jobAD.txt   11
4766_2021JAN_jobAD.txt   9
4767_2021JAN_jobAD.txt   26
4770_2021JAN_jobAD.txt   1
4772_2021JAN_jobAD.txt   5
4773_2021JAN_jobAD.txt   3
4780_2021JAN_jobAD.txt   10
4781_2021JAN_jobAD.txt   24
4785_2021JAN_jobAD.txt   11
4791_2021JAN_jobAD.txt   1
4799_2021JAN_jobAD.txt   6
4800_2021JAN_jobAD.txt   14
4803_2021JAN_jobAD.txt   2
4807_2021JAN_jobAD.txt   32
4809_2021JAN_jobAD.txt   2
4811_2021JAN_jobAD.txt   6
4813_2021JAN_jobAD.txt   8
4816_2021JAN_jobAD.txt   1
4820_2021JAN_jo

5720_2021JAN_jobAD.txt   11
5745_2021JAN_jobAD.txt   3
5746_2021JAN_jobAD.txt   1
5750_2021JAN_jobAD.txt   2
5751_2021JAN_jobAD.txt   14
5760_2021JAN_jobAD.txt   3
5763_2021JAN_jobAD.txt   4
5765_2021JAN_jobAD.txt   207
5766_2021JAN_jobAD.txt   2
5769_2021JAN_jobAD.txt   11
5776_2021JAN_jobAD.txt   5
5782_2021JAN_jobAD.txt   9
5783_2021JAN_jobAD.txt   4
5785_2021JAN_jobAD.txt   31
5789_2021JAN_jobAD.txt   4
5795_2021JAN_jobAD.txt   3
5809_2021JAN_jobAD.txt   11
5810_2021JAN_jobAD.txt   5
5811_2021JAN_jobAD.txt   15
5815_2021JAN_jobAD.txt   2
5821_2021JAN_jobAD.txt   3
5823_2021JAN_jobAD.txt   34
5831_2021JAN_jobAD.txt   11
5833_2021JAN_jobAD.txt   11
5834_2021JAN_jobAD.txt   4
5842_2021JAN_jobAD.txt   5
5843_2021JAN_jobAD.txt   11
5844_2021JAN_jobAD.txt   10
5848_2021JAN_jobAD.txt   2
5849_2021JAN_jobAD.txt   2
5851_2021JAN_jobAD.txt   9
5859_2021JAN_jobAD.txt   1
5861_2021JAN_jobAD.txt   3
5867_2021JAN_jobAD.txt   45
5869_2021JAN_jobAD.txt   7
5870_2021JAN_jobAD.txt   12
5872_2021JAN_

6727_2021JAN_jobAD.txt   14
6728_2021JAN_jobAD.txt   1
6734_2021JAN_jobAD.txt   10
6735_2021JAN_jobAD.txt   23
6737_2021JAN_jobAD.txt   7
6738_2021JAN_jobAD.txt   61
6742_2021JAN_jobAD.txt   13
6743_2021JAN_jobAD.txt   2
6745_2021JAN_jobAD.txt   11
6753_2021JAN_jobAD.txt   633
6756_2021JAN_jobAD.txt   18
6759_2021JAN_jobAD.txt   8
6760_2021JAN_jobAD.txt   5
6761_2021JAN_jobAD.txt   44
6763_2021JAN_jobAD.txt   20
6768_2021JAN_jobAD.txt   12
6772_2021JAN_jobAD.txt   11
6775_2021JAN_jobAD.txt   207
6776_2021JAN_jobAD.txt   9
6778_2021JAN_jobAD.txt   10
6789_2021JAN_jobAD.txt   11
6792_2021JAN_jobAD.txt   2
6794_2021JAN_jobAD.txt   3
6795_2021JAN_jobAD.txt   6
6796_2021JAN_jobAD.txt   63
6797_2021JAN_jobAD.txt   20
6801_2021JAN_jobAD.txt   3
6802_2021JAN_jobAD.txt   28
6805_2021JAN_jobAD.txt   8
6807_2021JAN_jobAD.txt   8
6809_2021JAN_jobAD.txt   7
6810_2021JAN_jobAD.txt   4
[1, 11, 21, 22, 23, 27, 28, 39, 41, 45, 49, 53, 54, 55, 56, 62, 65, 74, 84, 87, 91, 96, 108, 115, 116, 117, 121, 124

In [ ]:
#ITJobsList = []
#for i in range(0, len(urlList)):
#    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
#    if (os.path.exists(filename)):
#        with open(filename, encoding='utf-8') as fin:
#            all_ITs = re.findall(r"(?=("+'|'.join(itList)+r"))", fin.read())
#            nITs = len(all_ITs)
#            if nITs > 0:
#                print('tech? ',str(i+1)+"_2021JAN_jobAD.txt",' ',nITs)
#                ITJobsList.append(i+1)
#            else:
#                continue
#    else:
#        continue
        
#print(ITJobsList)

In [ ]:
#USAITJobsList = []
#fout = open("./ads/2021JAN_USAITjobAD.txt","w")
#for i in range(0, len(ITJobsList)):
#    filename = "./dat/"+str(ITJobsList[i])+"_2021JAN_jobAD.txt"
#    if (os.path.exists(filename)):
#        with open(filename, encoding='utf-8') as fin:
#            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
#            nUSAs = len(all_USAs)
#            if nUSAs > 0:
#                print('USA tech? ',str(ITJobsList[i])+"_2021JAN_jobAD.txt",' ',nUSAs)
#                fout.write(str(ITJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
#                USAITJobsList.append(ITJobsList[i])
#            else:
#                continue
#    else:
#        continue
            
#fout.close()
#print(USAITJobsList)

In [17]:
fout = open("./ads/2021JAN_USAQAs.txt","w")
for i in range(0, len(USAJobsList)):
    filename = "./dat/"+str(USAJobsList[i])+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            all_QAs = re.findall(r"(?=("+'|'.join(qaList)+r"))", fin.read())
            nQAs = len(all_QAs)
            if nQAs > 0:
                print('USA QAs? ',str(USAJobsList[i])+"_2021JAN_jobAD.txt",' ',nQAs)
                fout.write(str(USAJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nQAs)+'\n')
            else:
                continue
    else:
        continue
            
fout.close()

USA QAs?  27_2021JAN_jobAD.txt   1
USA QAs?  28_2021JAN_jobAD.txt   2
USA QAs?  96_2021JAN_jobAD.txt   2
USA QAs?  129_2021JAN_jobAD.txt   1
USA QAs?  221_2021JAN_jobAD.txt   2
USA QAs?  285_2021JAN_jobAD.txt   1
USA QAs?  293_2021JAN_jobAD.txt   1
USA QAs?  312_2021JAN_jobAD.txt   1
USA QAs?  362_2021JAN_jobAD.txt   1
USA QAs?  410_2021JAN_jobAD.txt   2
USA QAs?  453_2021JAN_jobAD.txt   1
USA QAs?  539_2021JAN_jobAD.txt   3
USA QAs?  575_2021JAN_jobAD.txt   1
USA QAs?  578_2021JAN_jobAD.txt   3
USA QAs?  610_2021JAN_jobAD.txt   3
USA QAs?  625_2021JAN_jobAD.txt   11
USA QAs?  629_2021JAN_jobAD.txt   1
USA QAs?  641_2021JAN_jobAD.txt   2
USA QAs?  681_2021JAN_jobAD.txt   3
USA QAs?  683_2021JAN_jobAD.txt   1
USA QAs?  763_2021JAN_jobAD.txt   1
USA QAs?  783_2021JAN_jobAD.txt   2
USA QAs?  786_2021JAN_jobAD.txt   1
USA QAs?  798_2021JAN_jobAD.txt   1
USA QAs?  807_2021JAN_jobAD.txt   1
USA QAs?  809_2021JAN_jobAD.txt   3
USA QAs?  810_2021JAN_jobAD.txt   8
USA QAs?  811_2021JAN_jobAD.tx

USA QAs?  4947_2021JAN_jobAD.txt   1
USA QAs?  4959_2021JAN_jobAD.txt   1
USA QAs?  4970_2021JAN_jobAD.txt   4
USA QAs?  4984_2021JAN_jobAD.txt   1
USA QAs?  5008_2021JAN_jobAD.txt   1
USA QAs?  5013_2021JAN_jobAD.txt   1
USA QAs?  5057_2021JAN_jobAD.txt   1
USA QAs?  5059_2021JAN_jobAD.txt   1
USA QAs?  5109_2021JAN_jobAD.txt   4
USA QAs?  5113_2021JAN_jobAD.txt   1
USA QAs?  5129_2021JAN_jobAD.txt   2
USA QAs?  5152_2021JAN_jobAD.txt   1
USA QAs?  5166_2021JAN_jobAD.txt   1
USA QAs?  5180_2021JAN_jobAD.txt   3
USA QAs?  5185_2021JAN_jobAD.txt   1
USA QAs?  5189_2021JAN_jobAD.txt   1
USA QAs?  5202_2021JAN_jobAD.txt   1
USA QAs?  5208_2021JAN_jobAD.txt   2
USA QAs?  5254_2021JAN_jobAD.txt   1
USA QAs?  5315_2021JAN_jobAD.txt   2
USA QAs?  5324_2021JAN_jobAD.txt   4
USA QAs?  5408_2021JAN_jobAD.txt   1
USA QAs?  5423_2021JAN_jobAD.txt   2
USA QAs?  5458_2021JAN_jobAD.txt   2
USA QAs?  5459_2021JAN_jobAD.txt   2
USA QAs?  5467_2021JAN_jobAD.txt   3
USA QAs?  5541_2021JAN_jobAD.txt   1
U